In [1]:
import pandas as pd
import joblib
import nltk
from nltk.tokenize import sent_tokenize
from bert_score import score

nltk.download("punkt")


C:\Users\Kabi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Kabi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [26]:
# Load the scientific test dataset (must have 'article' and 'summary' columns)
df = pd.read_csv("sci_test.csv")
df = df.dropna(subset=["text", "summary"]).reset_index(drop=True)
df_sample = df.sample(n=20, random_state=42)


In [27]:
# Load trained Random Forest model and TF-IDF vectorizer
model = joblib.load("random_forest_summary_model.joblib")
vectorizer = joblib.load("tfidf_vectorizer.joblib")


In [28]:
# Define extractive summarization function using predicted probabilities
def summarize_with_rf(text, model, vectorizer, top_k=3):
    sentences = sent_tokenize(text)
    if not sentences:
        return ""
    
    X = vectorizer.transform(sentences)
    probs = model.predict_proba(X)[:, 1]

    top_indices = probs.argsort()[::-1][:top_k]
    selected = [sentences[i] for i in sorted(top_indices)]
    return " ".join(selected)


In [29]:
# Generate summaries and prepare for evaluation
generated = []
references = []

for _, row in df_sample.iterrows():
    ref = row['summary']
    article = row['text']
    summary = summarize_with_rf(article, model, vectorizer, top_k=3)
    
    if summary.strip():
        generated.append(summary)
        references.append(ref)


In [30]:
# Evaluate using BERTScore
P, R, F1 = score(generated, references, lang="en", verbose=True)

print("\n🧪 BERTScore Results for Extractive Summarizer:")
print(f"Precision: {P.mean():.4f}")
print(f"Recall:    {R.mean():.4f}")
print(f"F1 Score:  {F1.mean():.4f}")


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


100%|██████████| 1/1 [00:17<00:00, 17.87s/it]


computing greedy matching.


100%|██████████| 1/1 [00:00<00:00, 56.86it/s]

done in 17.89 seconds, 1.12 sentences/sec

🧪 BERTScore Results for Extractive Summarizer:
Precision: 0.8388
Recall:    0.8313
F1 Score:  0.8348


In [31]:
# Save generated and reference summaries to a CSV file
summary_df = pd.DataFrame({
    "article": df_sample["text"][:len(generated)],
    "reference_summary": references,
    "generated_summary": generated
})

summary_df.to_csv("rf_extractive_summary_results_sci.csv", index=True)
print("✅ Summaries saved to rf_extractive_summary_results_sci.csv")


✅ Summaries saved to rf_extractive_summary_results_sci.csv


## Test the summarizer with news article dataset

In [18]:
df1 = pd.read_csv("news_test.csv")
df1= df1.dropna(subset=["text", "summary"]).reset_index(drop=True)
df_sample1 = df1.sample(n=20, random_state=42)

In [22]:
# Generate summaries and prepare for evaluation
generated1 = []
references1 = []

for _, row in df_sample1.iterrows():
    ref = row['summary']
    article = row['text']
    summary = summarize_with_rf(article, model, vectorizer, top_k=3)
    
    if summary.strip():
        generated1.append(summary)
        references1.append(ref)


In [23]:
# Evaluate using BERTScore
P_1, R_1, F1_1 = score(generated1, references1, lang="en", verbose=True)

print("\n🧪 BERTScore Results for Extractive Summarizer:")
print(f"Precision: {P_1.mean():.4f}")
print(f"Recall:    {R_1.mean():.4f}")
print(f"F1 Score:  {F1_1.mean():.4f}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


100%|██████████| 1/1 [00:20<00:00, 20.97s/it]


computing greedy matching.


100%|██████████| 1/1 [00:00<00:00, 46.81it/s]

done in 21.00 seconds, 0.95 sentences/sec

🧪 BERTScore Results for Extractive Summarizer:
Precision: 0.9012
Recall:    0.8671
F1 Score:  0.8837


In [32]:
# Save generated and reference summaries to a CSV file
summary_df1 = pd.DataFrame({
    "article": df_sample1["text"][:len(generated1)],
    "reference_summary": references1,
    "generated_summary": generated1
})

summary_df1.to_csv("rf_extractive_summary_results_news.csv", index=True)
print("✅ Summaries saved to rf_extractive_summary_results_news.csv")

✅ Summaries saved to rf_extractive_summary_results_news.csv
